In [ ]:
# Import used packages
%matplotlib notebook
import numpy as np
from scipy import spatial, optimize
import matplotlib.pyplot as plt
import random
from matplotlib import animation
import time

In [ ]:
# Create positions in rectangle of xlen x ylen
def randompos(nodes, xLen, yLen):
    pos = []
    for i in range(nodes):
        pos.append([xLen*random.uniform(0, 1), yLen*random.uniform(0, 1)])
    return pos

# Link nodes using KDtrees
def linknodesKDtree(pos, radius):
    treePos = spatial.KDTree(pos)
    links = treePos.query_ball_tree(other= treePos, r= radius)
    for i in range(len(links)):
        links[i].remove(i)
    return links

# Create network in a dictionary structure
def createnetwork(nodes, radius, xLen, yLen):
    Network = {'nodes': nodes, 'pos': randompos(nodes, xLen, yLen)}
    Network['links'] = linknodesKDtree(Network['pos'], radius)
    return Network

# plot nodes
def plotnodes(pos, ax, sizeNodes, colorNodes):
    ax.scatter([item[0] for item in pos], [item[1] for item in pos], s= sizeNodes, c= colorNodes, zorder= 2)
    
#plot edges
def plotedges(pos, ax, links, colorEdges, alphaEdges):
    xList = []
    yList = []
    for i in range(len(links)):
        for node in links[i]:
            if node > i:
                xList += [pos[i][0]]
                yList += [pos[i][1]]
                xList += [pos[node][0]]
                yList += [pos[node][1]]
                xList += [np.nan]
                yList += [np.nan]
    ax.plot(xList, yList, color = colorEdges, alpha= alphaEdges, zorder= 1)
    
# Plot a random graph
def plotrandomgraph(nodes, radius, xlen, ylen, sizeNodes, colorNodes, colorEdges, alphaEdges):
    Network = createnetwork(nodes, radius, xlen, ylen)
    fig, ax = plt.subplots(figsize = [6*xlen, 6*ylen])
    plt.axis('off')
    plotedges(Network['pos'], ax, Network['sets'], colorEdges, alphaEdges)
    plotnodes(Network['pos'], ax, sizeNodes, colorNodes)

In [ ]:
# remove a fraction of removePerc links from the network
def removelinks(links, removePerc):
    removeN = round(len(links)*removePerc)
    for i in range(removeN):
        randomNode = random.choice(random.choices(links, weights= map(len, links), k= 1))
        randomConnectedNode = random.choice(links[randomNode])
        links[randomNode].remove(randomConnectedNode)
        links[randomConnectedNode].remove(randomNode)
    return links

In [ ]:
radius = np.sqrt(2)
timeList = []
stepList = [32, 64, 128, 256, 512, 1024, 2048, 4096, 8192]
for i in stepList:
    startTime = time.time()
    for j in range(3):
        createnetwork(i, radius, 1, 1)
    timeList.append((time.time() - startTime)/5)
    print(i)
timeList

In [ ]:
def kwadratic(x, a, b, c):
    return a*x**2 + b*x + c

par, parCov = optimize.curve_fit(function, stepList, timeList)
par

In [ ]:
fig = plt.figure()
functionList = np.arange(0, 10000, 10)
plt.plot(stepList, timeList, 'o', label= 'Data')
plt.plot(functionList, kwadratic(functionList, par[0], par[1], par[2]))
plt.legend()
plt.show()
# Kan dit wel sneller dan O(n**2)? van n nodes naar 2n nodes betekend van n-1+n-2+... = (n-1)(n-2)/2 edges naar 2n-1+2n-2+... = (2n-1)(2n-2)/2 edges -> O(n**2)